In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a SparkSession
spark = SparkSession.builder.appName("App").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

## Load CSV File with Header

In [4]:
# Load the CSV file with the first row as a header
df = spark.read.format("csv").option("delimiter", ",").option("header", "true").load("1987.csv")

# Display the columns and the first 15 rows
df.show(15, truncate=False)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1987|10   |14        |3        |741    |730       |912    |849       |PS           |1451     

In [5]:
# Iterate over all columns in the DataFrame
for column in df.columns:
    df = df.withColumn(column, when(col(column) == "NA", None).otherwise(col(column)))

df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1987|   10|        14|        3|    741|       730|    912|       849|           PS|     1451

In [6]:
from pyspark.sql.functions import col

# Counts the number of null values for each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance| TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|   0|    0|         0|        0|  19685|         0|  23500|         0|            0|      

# Preprocessing
- eliminate unnecesary variables
- missing and duplicates values
- see correlation
- variable transformation
- variable creation

In [22]:
# columns to eliminate
columns = [
    "ArrTime", 
    "ActualElapsedTime", 
    "AirTime", 
    "TaxiIn", 
    "Diverted", 
    "CarrierDelay", 
    "WeatherDelay", 
    "NASDelay", 
    "SecurityDelay", 
    "LateAircraftDelay"
]

# Eliminate columns
df = df.drop(*columns)


In [7]:
# columns to eliminate
columns = [
    "Year",
    "TailNum",
    "TaxiOut",
    "Cancelled",
    "CancellationCode"  
]

# Eliminate columns
df = df.drop(*columns)

## Missing values

In [8]:
# Contar valores NA por columna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance| TaxiIn|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+-------+--------+------------+------------+--------+-------------+-----------------+
|    0|         0|        0|  19685|         0|  23500|         0|            0|        0|            23500|             0|1311826|   23500|   19685|     0|   0|    1015|1311826|       0|     1311826|     1311826| 1311826|      1311826

#Now we check if NA stands for 0. If this value is not present, means that NA was 0.

In [9]:
from pyspark.sql import functions as F
# Filter the DataFrame to keep only rows where ArrDelay is equal to 0
filtered_df = df.filter(F.col("ArrDelay") == 0)

# Total number of rows in the DataFrame
total_rows = df.count()

# Check if there are any rows in the filtered DataFrame
if filtered_df.count() > 0:
    print("0 is present in the ArrDelay column " + str(filtered_df.count()) + " times out of " + str(total_rows) + ".")
else:
    print("0 is not present in the ArrDelay column.")

0 is present in the ArrDelay column 60436 times out of 1311826.


In [10]:
# Calculate the percentage of null values for each column
null_percentage = df.select([(count(when(col(c).isNull(), c)) / total_rows).alias(c) for c in df.columns])

# Show the percentage of null values for each column
null_percentage.show()

+-----+----------+---------+--------------------+----------+--------------------+----------+-------------+---------+--------------------+--------------+-------+--------------------+--------------------+------+----+--------------------+------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DayOfWeek|             DepTime|CRSDepTime|             ArrTime|CRSArrTime|UniqueCarrier|FlightNum|   ActualElapsedTime|CRSElapsedTime|AirTime|            ArrDelay|            DepDelay|Origin|Dest|            Distance|TaxiIn|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+---------+--------------------+----------+--------------------+----------+-------------+---------+--------------------+--------------+-------+--------------------+--------------------+------+----+--------------------+------+--------+------------+------------+--------+-------------+-----------------+
|  0.0|       0.0|      0.0|0.0150058010742

In [11]:
# Drop rows with at least one missing value

df = df.dropna()
dropped_rows = total_rows - df.count()
print("Dropped "+ str(dropped_rows)+ " rows.")

Dropped 1311826 rows.


## Duplicates

In [12]:
# Check for duplicates and show the results
total_rows = df.count()
df = df.dropDuplicates()

if total_rows - df.count()  > 0:
    print("There are duplicates in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

No duplicates found in the DataFrame.


## Variable transformation

In [13]:
# List of columns to exclude from conversion
exclude_columns = ['UniqueCarrier', 'Origin', 'Dest']

# Convert all columns to integer type except the ones in exclude_columns
for column in df.columns:
    if column not in exclude_columns:
        df = df.withColumn(column, col(column).cast("integer"))

# Display the columns and the first 15 rows to verify the change
df.show(15, truncate=False)



+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+------+--------+------------+------------+--------+-------------+-----------------+
+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-----------------+--------------+-------+--------+--------+------+----+--------+------+--------+------------+------------+--------+-------------+---

In [14]:
# Print distinct values for each specified column
for column in exclude_columns:
    print(f"Distinct values in column '{column}':")
    distinct_values = df.select(column).distinct().collect()
    for value in distinct_values:
        print(value[column])
    print("\n")  # Adding a newline for better readability
    # Print the number of elements in the distinct_values list
    print(f"Number of distinct values: {len(distinct_values)}")

Distinct values in column 'UniqueCarrier':


Number of distinct values: 0
Distinct values in column 'Origin':


Number of distinct values: 0
Distinct values in column 'Dest':


Number of distinct values: 0


## Import geographic coordinates

In [15]:
# Read the CSV file
usa_airport_df = spark.read.csv("us-airports.csv", header=True)

# Select only the 'latitude_deg', 'longitude_deg', and 'iata_code' columns
usa_airport_df = usa_airport_df.select("latitude_deg", "longitude_deg", "iata_code")

usa_airport_df.show()

+------------------+------------------+--------------------+
|      latitude_deg|     longitude_deg|           iata_code|
+------------------+------------------+--------------------+
|         #geo +lat|         #geo +lon|#loc +airport +co...|
|         33.942501|       -118.407997|                 LAX|
|           41.9786|          -87.9048|                 ORD|
|         40.639447|        -73.779317|                 JFK|
|           33.6367|        -84.428101|                 ATL|
| 37.61899948120117|          -122.375|                 SFO|
|         40.692501|        -74.168701|                 EWR|
|         32.896801|        -97.038002|                 DFW|
|         36.083361|       -115.151817|                 LAS|
|28.429399490356445|-81.30899810791016|                 MCO|
|   39.861698150635|    -104.672996521|                 DEN|
|           38.9445|        -77.455803|                 IAD|
|         40.777199|        -73.872597|                 LGA|
| 25.79319953918457|-80.

In [16]:
# Select and concatenate the values from 'Origin' and 'Destination' columns
origin = df.select("Origin").distinct()
destination = df.select("Dest").distinct()

all_airports = origin.distinct().union(destination)

# Get distinct values
distinct_airports = all_airports.distinct()
print(all_airports.count())



# Collect the distinct values

#Distinct airport codes in the new Dataset.
usa_iata_codes = usa_airport_df.select("iata_code").distinct()


#Show the missing values

distinct_airports.subtract(usa_iata_codes).show()


472
+------+
|Origin|
+------+
|   SPN|
|   UCA|
|   SJU|
|   YAP|
|   STX|
|   PFN|
|   GUM|
|   ROR|
|   STT|
+------+



In [17]:
import re

# Function to convert DMS (degrees, minutes, seconds) to DD (decimal degrees)
def dms_to_dd(dms):
    parts = re.split('[°′″]', dms)
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3]
    
    dd = degrees + minutes/60 + seconds/3600
    if direction in ('S', 'W'):
        dd *= -1
    return dd

# Coordinates in DMS format with corresponding  codes
coordinates = [
    ("43°08′36″N", "075°22′48″W","UCA"),
    ("18°20′14″N", "064°58′24″W","STT"),
    ("7°22′02″N", "134°32′39″E","ROR"),
    ("17°42′16″N", "64°48′06″W","STX"),
    ("9°29′56″N", "138°04′57″E","YAP"),
    ("13°29′02″N", "144°47′50″W","GUM"),
    ("15°07′08″N", "145°43′46″E","SPN"),
    ("30°12′44″N", "085°40′58″W","PFN"),
    ("18°26′22″N", "66°00′07″W","SJU")
]

# Convert the DMS coordinates to decimal degrees
airport_coordinates_dd = [(dms_to_dd(lat), dms_to_dd(lon),code) for  lat, lon ,code in coordinates]

new_airports_df = spark.createDataFrame(airport_coordinates_dd, ["latitude_deg", "longitude_deg", "iata_code"])
new_airports_df.show()

usa_airport_df = usa_airport_df.union(new_airports_df).distinct()

Py4JJavaError: An error occurred while calling o630.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 68.0 failed 1 times, most recent failure: Lost task 0.0 in stage 68.0 (TID 172) (Claudia.technicolor.net executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:3

In [ ]:
#Check that now we have coordinates for each airport
distinct_airports.subtract(usa_airport_df.select("iata_code")).show()

In [ ]:
# Now we merge the two dataframes
flights_with_origin = df.join(usa_airport_df, df["Origin"] == usa_airport_df["iata_code"])
flights_with_origin = flights_with_origin.withColumnRenamed("latitude_deg", "Origin_Lat").withColumnRenamed("longitude_deg", "Origin_Long")

#flights_with_origin.show()


In [ ]:
#adding destination coordinates
usa_airport_df = usa_airport_df.distinct()
flights_with_dest = flights_with_origin.join(usa_airport_df, flights_with_origin["Dest"] == usa_airport_df["iata_code"])
flights_with_dest = flights_with_dest.withColumnRenamed("latitude_deg", "Dest_Lat").withColumnRenamed("longitude_deg", "Dest_Long")

#flights_with_dest.show()


In [ ]:
#Now we have the coordinates of the airports instead of the codes.

merged_df = flights_with_dest.drop("iata_code","Origin","Dest")

#merged_df.show()

## Use OneHotEncoder for UniqueCarrier

In [55]:
#Now we transform the 'UniqueCarrrier' feature in a OneHotEncoder

from pyspark.ml.feature import StringIndexer, OneHotEncoder
#from pyspark.ml import Pipeline

# Create a StringIndexer
indexer = StringIndexer(inputCol="UniqueCarrier", outputCol="UniqueCarrierIndex")

# Fit the indexer to the DataFrame and transform it
df_indexed = indexer.fit(merged_df).transform(merged_df)

# Create a OneHotEncoder
encoder = OneHotEncoder(inputCols=["UniqueCarrierIndex"], outputCols=["UniqueCarrierVec"])

# Apply the encoder to the DataFrame
df_encoded = encoder.fit(df_indexed).transform(df_indexed)
df_encoded.select("UniqueCarrier", "UniqueCarrierVec").distinct().show(truncate=False)


NameError: name 'merged_df' is not defined

In [ ]:
#Print Results
df_encoded.select("UniqueCarrier", "UniqueCarrierVec").distinct().show(truncate=False)

In [ ]:
final_df = df_encoded.drop("UniqueCarrier")


# Convert "Origini_Lat" from string to double
final_df = final_df.withColumn("Origin_Lat", final_df["Origin_Lat"].cast("double"))

# Convert "Origini_Long" from string to double
final_df = final_df.withColumn("Origin_Long", final_df["Origin_Long"].cast("double"))

# Convert "Dest_Lat" from string to double
final_df = final_df.withColumn("Dest_Lat", final_df["Dest_Lat"].cast("double"))

# Convert "Dest_Long" from string to double
final_df = final_df.withColumn("Dest_Long", final_df["Dest_Long"].cast("double"))



## Plot to check the coordinates values

In [ ]:
# Plot the coordinates to inspect data
lat_long_df = final_df.select("Origin_Lat", "Origin_Long").toPandas()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# Taking a random sample of the data if it's too large
sample_df = lat_long_df.sample(frac=1)  # Adjust frac as needed

# Convert the 'Origin_Lat' and 'Origin_Long' columns to numeric (floats)
sample_df['Origin_Lat'] = pd.to_numeric(sample_df['Origin_Lat'], errors='coerce')
sample_df['Origin_Long'] = pd.to_numeric(sample_df['Origin_Long'], errors='coerce')

# Check the conversion
print(sample_df.dtypes)

# Now create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x=sample_df["Origin_Long"], y=sample_df["Origin_Lat"], alpha=1,s=2)
plt.title("Scatter Plot of Origin Coordinates")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()

In [ ]:
import sys
print(sys.executable)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrame is named sample_df and has been converted to numeric types

# Calculate Q1, Q3, and IQR for Latitude
Q1_lat = sample_df['Origin_Lat'].quantile(0.25)
Q3_lat = sample_df['Origin_Lat'].quantile(0.75)
IQR_lat = Q3_lat - Q1_lat

# Calculate Q1, Q3, and IQR for Longitude
Q1_long = sample_df['Origin_Long'].quantile(0.25)
Q3_long = sample_df['Origin_Long'].quantile(0.75)
IQR_long = Q3_long - Q1_long

# Define bounds for outliers
lower_bound_lat = Q1_lat - 1.5 * IQR_lat
upper_bound_lat = Q3_lat + 1.5 * IQR_lat
lower_bound_long = Q1_long - 1.5 * IQR_long
upper_bound_long = Q3_long + 1.5 * IQR_long

# Filter out outliers
filtered_df = sample_df[(sample_df['Origin_Lat'] >= lower_bound_lat) & 
                        (sample_df['Origin_Lat'] <= upper_bound_lat) &
                        (sample_df['Origin_Long'] >= lower_bound_long) & 
                        (sample_df['Origin_Long'] <= upper_bound_long)]

# Now create the scatter plot with smaller points and without outliers
plt.figure(figsize=(10, 6))
plt.scatter(x=filtered_df["Origin_Long"], y=filtered_df["Origin_Lat"], alpha=0.1, s=2)
plt.title("Scatter Plot of Origin Coordinates (Outliers Removed)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()


## Correlation

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# Select the numerical columns you want to include in the correlation analysis
numerical_cols = [col_name for col_name, data_type in final_df.dtypes if data_type == 'int' or data_type == 'double']



# Create a vector assembler to assemble the features
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features")
assembled_df = assembler.transform(final_df)

# Calculate the correlation matrix
corr_matrix = Correlation.corr(assembled_df, "features").head()[0]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Convert the correlation matrix to a Pandas DataFrame for visualization
corr_df = pd.DataFrame(corr_matrix.toArray(), columns=numerical_cols, index=numerical_cols)

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
#We drop some columns that are higly correlated with another

final_df = final_df.drop("CRSElapsedTime","CRSElapsedTime","CRSArrTime","CRSDepTime")

final_df.printSchema()

## Variable creation

In [15]:
from pyspark.sql.functions import when, col

# 1. Define classes based on ArrDelay
# Categorize flights as "Not Late" (0) if ArrDelay <= 0, and "Late" (1) otherwise
final_df = final_df.withColumn("ArrDelayClass", when(final_df["ArrDelay"] <= 0, 0).otherwise(1))

# Count the frequency of each class in the 'label' column
class_counts = final_df.groupBy("ArrDelayClass").count().orderBy("ArrDelayClass")

# Show the class counts
class_counts.show()

+-------------+-----+
|ArrDelayClass|count|
+-------------+-----+
+-------------+-----+



In [16]:
df_a = final_df.filter(df['ArrDelayClass'] == 1)
df_b = final_df.filter(df['ArrDelayClass'] == 0)

a_count = df_a.count()
b_count = df_b.count() 
ratio = a_count / b_count

df_b_oversampled = df_b.sample(withReplacement=True, fraction=ratio, seed=1)
combined_df = df_a.unionAll(df_b_oversampled)

# Count the frequency of each class in the 'ArrDelayClass' column after oversampling
class_counts = combined_df.groupBy("ArrDelayClass").count().orderBy("ArrDelayClass")

# Show the class counts
class_counts.show()

ZeroDivisionError: division by zero

In [44]:
combined_df.show()

+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+-------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|ArrDelayClass|
+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+-------------+
|   10|        10|        6|   1635|      1635|      1732|           PS|     1454|            57|       8|       0|   OAK| BUR|     325|            1|
|   10|        20|        2|   1945|      1945|      2101|           PS|     1484|            76|       1|       0|   SFO| SAN|     447|            1|
|   10|         9|        5|   2057|      2016|      2132|           PS|     1486|            76|      47|      41|   OAK| SAN|     446|            1|
|   10|        13|        2|   2114|      2115|      2223|           PS|     1505|            

# Modeling

In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

ModuleNotFoundError: No module named 'distutils'

In [ ]:
df.printSchema()

In [45]:
# Assuming your target variable is "ArrDelay" and your feature columns are selected
feature_columns = ["Month", "DayofMonth", "DayOfWeek", "DepTime", "FlightNum", "DepDelay", "UniqueCarrierIndex"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(final_df)

IllegalArgumentException: UniqueCarrierIndex does not exist. Available: Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, CRSArrTime, UniqueCarrier, FlightNum, CRSElapsedTime, ArrDelay, DepDelay, Origin, Dest, Distance, ArrDelayClass

In [ ]:
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=123)

In [ ]:
lr = LinearRegression(featuresCol="features", labelCol="ArrDelay",regParam=0.01)
lr_model = lr.fit(train_data)


In [ ]:
test_results = lr_model.evaluate(test_data)

# Print evaluation metrics
print("Root Mean Squared Error (RMSE):", test_results.rootMeanSquaredError)
print("R-squared (R2):", test_results.r2)

In [17]:
from pyspark.ml.classification import RandomForestClassifier
#from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.ml.evaluation.MulticlassClassificationEvaluator

# RandomForestClassifier
# Assuming your target variable is "ArrDelay" and your feature columns are selected.drop([]

columns = [
"ArrDelay"
]

# Eliminate columns
combined_df = combined_df.drop(*columns)

feature_columns = ["Month", "DayofMonth", "DayOfWeek", "DepTime", "FlightNum", "DepDelay", "UniqueCarrierIndex"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(combined_df)
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=123)
# set rf model 
rf = RandomForestClassifier(labelCol="ArrDelayClass", featuresCol="features")

# instantiate pipeline
#pipeline = Pipeline(stages=[assembled_data, rf])

# train model
model_rf = rf.fit(train_data)

# create prediction column on test data
results = model_rf.transform(test_data)

# evaluate results
# Define the hyperparameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

# Create the cross-validator
cross_validator = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="ArrDelayClass", metricName="accuracy"),
                          numFolds=5, seed=42)

# Train the model with the best hyperparameters
cv_model = cross_validator.fit(train_data)

best_rf_model = cv_model.bestModel.stages[-1]
importances = best_rf_model.featureImportances


print("Feature Importances:")
for feature, importance in zip(feature_columns, importances):
    print(f"{feature}: {importance:.4f}")

# Make predictions on the test data
predictions = cv_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="ArrDelayClass", metricName="accuracy")

# Evaluate the model
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = {:.2f}".format(accuracy))
    

ModuleNotFoundError: No module named 'distutils'

# Validation

##Close the context

In [ ]:
spark.stop()